In [1]:
import pandas as pd
import math

In [2]:
pulsar_df = pd.read_csv('final.csv')
star = { 'MIP': 0.533, 'STDIP': 0.574, 'EKIP': 0.172, 'SIP': 0.011, 'MC': 0.35, 'STDC': 0.695, 'EKC': 0.483, 'SC': 0.179 }
pulsar_df

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
0,76.523438,34.087196,2.024270,8.131775,7.983278,36.657363,4.888890,23.675354,1
1,98.367188,50.305919,0.409147,0.073141,0.842809,11.235635,16.359947,301.281519,0
2,98.468750,38.377634,0.444190,1.620170,2.318562,15.334935,10.598637,141.679457,0
3,73.601562,37.389282,1.978300,7.076543,5.881271,30.217483,5.758598,33.909779,1
4,98.390625,38.090606,0.181847,1.328353,2.646321,15.262458,8.322597,90.983960,0
...,...,...,...,...,...,...,...,...,...
198,98.398438,39.894263,0.355472,1.479615,4.084448,21.655104,6.472466,47.479061,0
199,98.203125,35.151093,0.615165,2.365818,2.602007,15.251530,8.414599,92.122652,0
200,98.398438,38.684893,0.309010,1.338891,3.319398,21.789852,8.000014,70.629042,0
201,98.195312,41.388038,0.494908,0.943453,13.322742,49.246271,3.451562,10.029465,0


In [3]:
pulsar_df.apply(pd.to_numeric)

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
0,76.523438,34.087196,2.024270,8.131775,7.983278,36.657363,4.888890,23.675354,1
1,98.367188,50.305919,0.409147,0.073141,0.842809,11.235635,16.359947,301.281519,0
2,98.468750,38.377634,0.444190,1.620170,2.318562,15.334935,10.598637,141.679457,0
3,73.601562,37.389282,1.978300,7.076543,5.881271,30.217483,5.758598,33.909779,1
4,98.390625,38.090606,0.181847,1.328353,2.646321,15.262458,8.322597,90.983960,0
...,...,...,...,...,...,...,...,...,...
198,98.398438,39.894263,0.355472,1.479615,4.084448,21.655104,6.472466,47.479061,0
199,98.203125,35.151093,0.615165,2.365818,2.602007,15.251530,8.414599,92.122652,0
200,98.398438,38.684893,0.309010,1.338891,3.319398,21.789852,8.000014,70.629042,0
201,98.195312,41.388038,0.494908,0.943453,13.322742,49.246271,3.451562,10.029465,0


### Выборочное среднее для столбца MIP

In [4]:
round(pulsar_df.MIP.mean(), 3)

86.3

### Определите выборочное среднее для столбца MIP после нормировки

In [5]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(pulsar_df)
min_max_array = min_max_scaler.transform(pulsar_df)

In [6]:
min_max_df = pd.DataFrame(min_max_array, columns=['MIP', 'STDIP', 'EKIP', 'SIP', 'MC', 'STDC', 'EKC', 'SC', 'TARGET'])

In [7]:
round(min_max_df.MIP.mean(),3)

0.561

### Обучите модель логистической регрессии, используя полученную после нормировки выборку в качестве тренировочного набора данных. 

In [23]:
train_data = min_max_df

In [24]:
X = pd.DataFrame(train_data.drop(['TARGET'], axis=1))
y = pd.DataFrame(train_data['TARGET'])

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y.values.ravel())

In [27]:
reg.predict_proba([[star['MIP'], star['STDIP'], star['EKIP'], star['SIP'], star['MC'], star['STDC'], star['EKC'], star['SC']]])

array([[0.56713988, 0.43286012]])

### Выполните классификацию новой звезды, с помощью метода -ближайших соседей, используя нормализованные данные выборки.

In [13]:
distances = []
for mip, stdip, ekip, sip, mc, stdc, eks, sc in zip(min_max_df.MIP, min_max_df.STDIP, min_max_df.EKIP, min_max_df.SIP, min_max_df.MC, min_max_df.STDC, min_max_df.EKC, min_max_df.SC):
    distance = math.sqrt((mip - star['MIP'])**2 + (stdip - star['STDIP'])**2 + (ekip - star['EKIP'])**2 + (sip - star['SIP'])**2 + (mc - star['MC'])**2 + (stdc - star['STDC'])**2 + (eks - star['EKC'])**2 + (sc - star['SC'])**2)
    distances.append(distance)

In [14]:
distances_df = pd.DataFrame(distances, columns=['Distance'])
distances_df.sort_values(by=['Distance'])

,Distance
89,0.633082
41,0.648111
117,0.674377
130,0.688011
19,0.691521
...,...
116,1.386811
145,1.402973
121,1.421466
156,1.452561


In [15]:
pulsar_df['Distance'] = distances_df
pulsar_df.sort_values(by=['Distance'])

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET,Distance
89,76.218750,46.775718,1.133907,1.858365,50.635452,77.106424,1.185711,-0.176323,1,0.633082
41,77.007812,47.940474,1.236140,1.925760,22.075251,55.060496,2.468206,4.924129,1,0.648111
117,74.632812,44.241149,0.996855,1.839553,32.307692,62.667360,1.903620,2.378535,1,0.674377
130,98.226562,44.639216,0.742024,0.989412,33.214883,67.032637,1.753289,1.541359,0,0.688011
19,76.054688,60.412942,1.432917,0.936654,12.502508,42.661566,3.993995,15.720422,1,0.691521
...,...,...,...,...,...,...,...,...,...,...
116,71.585938,33.705185,2.816974,12.347815,30.444816,70.394893,2.103614,2.836093,1,1.386811
145,72.656250,34.347816,2.819005,11.731853,7.459030,33.433919,5.142988,27.548528,1,1.402973
121,73.515625,30.018740,2.679638,12.548989,6.854515,34.946384,5.515590,30.322017,1,1.421466
156,71.781250,30.018988,2.776656,13.222990,20.209866,52.927526,2.772906,6.809022,1,1.452561
